## Dependencies

In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from datetime import datetime

## Model

In [2]:
class ChatBot:
    def __init__(self, embeddings_path):
        self.word_vectors = KeyedVectors.load_word2vec_format(embeddings_path, binary=False, no_header=True)

        # preprocessing tools
        self.stop_words = set(stopwords.words("english"))
        self.lemmatizer = WordNetLemmatizer()

        # intent dictionary
        self.intents = {
            "greeting": ["hello", "hi", "hey", "good morning", "good evening", "how are you", "what's up"],
            "book_flight": ["book a flight", "make a reservation", "i want to fly", "flight booking", "reserve ticket"],
            "ask_flight_time": ["flight time", "when is my flight", "schedule", "departure time"],
            "cancel_reservation": ["cancel booking", "delete my reservation", "cancel my flight", "remove ticket"],
            "farewell": ["bye", "goodbye", "thank you", "thanks", "see you later", "take care"]
        }

        # reservation details frame
        self.reservation_details = {
            "destination": None,
            "departure_date": None,
            "return_date": None,
            "class": None,
            "confirmation": None
        }

    def preprocess_text(self, text):
        """
        Preprocess text: lowercase, tokenize, lemmatize, remove punctuation and stopwords.
        """
        text = text.lower().translate(str.maketrans("", "", string.punctuation))
        tokens = re.findall(r'\b\w+\b',text)
        return [self.lemmatizer.lemmatize(word) for word in tokens if word not in self.stop_words]

    def embed_sentence(self, sentence):
        tokens = self.preprocess_text(sentence)
        embeddings = [self.word_vectors[word] for word in tokens if word in self.word_vectors]
        if embeddings:
            return np.mean(embeddings, axis=0)
        return None

    def match_intent(self, user_input):
        input_embedding = self.embed_sentence(user_input)
        if input_embedding is None:
            return "unknown"

        best_intent = None
        best_similarity = 0

        for intent, examples in self.intents.items():
            for example in examples:
                example_embedding = self.embed_sentence(example)
                if example_embedding is not None:
                    similarity = cosine_similarity([input_embedding], [example_embedding])[0][0]
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_intent = intent

        return best_intent if best_similarity > 0.5 else "unknown"

    def validate_input(self, input_value, input_type, context=None):

        if input_type == "destination":
            if input_value.strip():
                return input_value
            print("TravelBot: Destination cannot be empty.")
            return self.validate_input(input("TravelBot: Where are you flying to? "), "destination")

        elif input_type == "date":
            try:
                date_obj = datetime.strptime(input_value, "%Y-%m-%d")
                if date_obj > datetime.now():
                    return input_value
                print("TravelBot: Please enter a future date.")
            except ValueError:
                print("TravelBot: Invalid date format. Use YYYY-MM-DD.")
            return self.validate_input(input("TravelBot: Enter a valid date (YYYY-MM-DD): "), "date")

        elif input_type == "return_date":
            try:
                dep_date = datetime.strptime(context, "%Y-%m-%d")
                ret_date = datetime.strptime(input_value, "%Y-%m-%d")
                if ret_date > dep_date:
                    return input_value
                print("TravelBot: Return date must be after the departure date.")
            except ValueError:
                print("TravelBot: Invalid date format. Use YYYY-MM-DD.")
            return self.validate_input(input("TravelBot: Enter a valid return date (YYYY-MM-DD): "), "return_date", context)

        elif input_type == "class":
            valid_classes = ["economy", "business"]
            if input_value.lower() in valid_classes:
                return input_value.title()
            print("TravelBot: Please enter either 'Economy' or 'Business'.")
            return self.validate_input(input("TravelBot: Which class do you prefer (Economy/Business)? "), "class")

    def handle_booking(self):

        print("TravelBot: Great! Let's book a flight. I'll ask you some questions.")

        self.reservation_details["destination"] = self.validate_input(
            input("TravelBot: Where are you flying to? "), "destination"
        )
        self.reservation_details["flying_from"] = self.validate_input(
            input("TravelBot: Where are you flying from? "), "destination"
        )
        self.reservation_details["departure_date"] = self.validate_input(
            input("TravelBot: What is your departure date (YYYY-MM-DD)? "), "date"
        )
        self.reservation_details["return_date"] = self.validate_input(
            input("TravelBot: What is your return date (YYYY-MM-DD)? "), "return_date", self.reservation_details["departure_date"]
        )
        self.reservation_details["class"] = self.validate_input(
            input("TravelBot: Which class do you prefer (Economy/Business)? "), "class"
        )

        print("TravelBot: Got it! Here are your reservation details:")
        self.print_reservation_details()

        self.reservation_details["confirmation"] = input("TravelBot: Do you want to confirm this reservation? (yes/no): ").lower()

        if self.reservation_details["confirmation"] == "yes":
            print("TravelBot: Your flight has been booked! Safe travels!")
        else:
            print("TravelBot: Your reservation was not confirmed. Let me know if you need further assistance.")


    def start_chat(self):
        print("TravelBot: Hello! I'm TravelBot. I can help you book flights, check flight times, and more.")
        while True:
            user_input = input("You: ")
            intent = self.match_intent(user_input)

            if intent == "greeting":
                print("TravelBot: Hi there! How can I assist you today?")
            elif intent == "book_flight":
                self.handle_booking()
            elif intent == "ask_flight_time":
                print("TravelBot: Please provide your reservation ID to check your flight time.")
            elif intent == "cancel_reservation":
                print("TravelBot: Your reservation has been canceled.")
            elif intent == "farewell":
                print("TravelBot: Goodbye! Have a great day!")
                break
            else:
                print("TravelBot: I'm sorry, I didn't understand that. Could you rephrase?")
    
    def print_reservation_details(self):
        print("\nReservation Details:")
        for key, value in self.reservation_details.items():
            if key != "confirmation" and value:
                print(f"\n  {key.replace('_', ' ').capitalize()}: {value}")

## Chat

In [ ]:
# load embeddings
embeddings_path = "./models/glove.6B.100d.txt"  # use your own embedding model path
# create ChatBot instance
chatbot = ChatBot(embeddings_path)

# start the chatbot interaction
chatbot.start_chat()

TravelBot: Hello! I'm TravelBot. I can help you book flights, check flight times, and more.
TravelBot: Hi there! How can I assist you today?
TravelBot: Great! Let's book a flight. I'll ask you some questions.
TravelBot: Got it! Here are your reservation details:

Reservation Details:

  Destination: Boston

  Departure date: 2024-12-12

  Return date: 2025-01-12

  Class: Economy

  Flying from: New york
TravelBot: Your flight has been booked! Safe travels!
TravelBot: Great! Let's book a flight. I'll ask you some questions.
TravelBot: Got it! Here are your reservation details:

Reservation Details:

  Destination: Buffalo

  Departure date: 2024-12-15

  Return date: 2025-01-11

  Class: Economy

  Flying from: Boston
TravelBot: Your flight has been booked! Safe travels!
TravelBot: Great! Let's book a flight. I'll ask you some questions.
TravelBot: Please enter a future date.
TravelBot: Please enter a future date.
TravelBot: Return date must be after the departure date.
TravelBot: Plea

### **Overview**

The ChatBot is a rule-based, interactive system designed to assist users with flight bookings and basic travel-related inquiries. At its core, it leverages pre-trained word embeddings and cosine similarity to identify user intents. By mapping user inputs to predefined intent categories, the ChatBot achieves flexibility and robustness in understanding natural language, even when phrased differently.

### **Functionality**

Intent identification is performed by embedding user inputs and intent examples into a shared vector space using pre-trained word embeddings. Each word in the input is tokenized, lemmatized, and stripped of stopwords and punctuation to ensure consistency and compatibility with the embeddings. These embeddings are averaged to generate a vector representation of the input.

To match the user input with predefined intents, the ChatBot calculates cosine similarity between the input's vector representation and those of predefined examples for each intent category. The intent with the highest similarity score above a specified threshold is selected. This method allows the ChatBot to understand inputs that may not match exactly with predefined examples but convey similar meanings.

The system dynamically interacts with users, providing structured prompts and summarizing reservations before confirmation.
